In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
[1,2,3,0,0,0,1,2]

43


In [2]:
from pathlib import Path
from src.dataset_readers.vctk import VCTK
import random
import ujson as json
import logging
import sys

DATASET_ROOT = Path('/home/lnan6257/work/dataset/speech/VCTK-Corpus')
PROJECT_ROOT = Path('/home/lnan6257/work/allennlp-speech')
serilization_dir = Path('./serilization_dir')
serilization_dir.mkdir(exist_ok=True)

In [3]:
def init_logger():
    logger = logging.getLogger()

    ch = logging.StreamHandler(sys.stdout)
    logger.addHandler(ch)
    logger.setLevel(logging.DEBUG)
    h = logging.FileHandler(str(serilization_dir / 'log.out'), mode='w')
    logger.addHandler(h)

    return logger
logger = init_logger()

In [ ]:
#code for generate tiny dataset from full
"""
dataset_train = DATASET_ROOT / 'generated' / 'dataset_list.train'
dataset_dev = DATASET_ROOT / 'generated' / 'dataset_list.dev'
dataset_test = DATASET_ROOT / 'generated' / 'dataset_list.test'
def generate_small(dataset):
    with open(dataset) as fh:
        data = json.load(fh)
    small_dataset_path = Path(str(dataset) + '.small')
    size = len(data)
    small_size = max(100, int(size/50))
    random.shuffle(data)
    with open(small_dataset_path, 'w') as fh:
        json.dump(data[:small_size], fh)
#generate_small(dataset_train)
generate_small(dataset_dev)
generate_small(dataset_test)
"""

In [5]:
dataset_train_path = DATASET_ROOT / 'generated' / 'dataset_list.train.small'
dataset_dev_path = DATASET_ROOT / 'generated' / 'dataset_list.dev.small'
dataset_test_path = DATASET_ROOT / 'generated' / 'dataset_list.test.small'
with open(dataset_train_path) as file:
    dataset_train = json.load(file)
with open(dataset_dev_path) as file:
    dataset_dev = json.load(file)
with open(dataset_test_path) as file:
    dataset_test = json.load(file)
def show_static(name, dataset):
    print(f'static for {name}')
    print(f'size = {len(dataset)}')
    print('-' * 100)
show_static('train', dataset_train)
show_static('dev', dataset_dev)
show_static('test', dataset_test)

static for train
size = 685
----------------------------------------------------------------------------------------------------
static for dev
size = 100
----------------------------------------------------------------------------------------------------
static for test
size = 103
----------------------------------------------------------------------------------------------------


In [6]:
from allennlp.common import Params
params_dict = {
    'dataset_reader':{
        'type': 'vctk',
        'cache_path': DATASET_ROOT / 'data-small.hdf5',
        'feature_name': 'mag',
        'audio_transformer': {
            'type': 'mag'
        }
    },
    'train_data_path': dataset_train_path,
    'validation_data_path': dataset_dev_path,
    'test_data_path': dataset_test_path,
}
params = Params(params_dict)

In [8]:
from allennlp.data import DatasetReader
def datasets_from_params(params: Params):
    """
    Load all the datasets specified by the config.
    """
    dataset_reader = DatasetReader.from_params(params.get('dataset_reader'))
    validation_dataset_reader_params = params.get("validation_dataset_reader", None)

    validation_and_test_dataset_reader: DatasetReader = dataset_reader
    if validation_dataset_reader_params is not None:
        logger.info("Using a separate dataset reader to load validation and test data.")
        validation_and_test_dataset_reader = DatasetReader.from_params(validation_dataset_reader_params)

    train_data_path = params.get('train_data_path')
    logger.info("Reading training data from %s", train_data_path)
    train_data = dataset_reader.read(train_data_path)

    datasets: Dict[str, Iterable[Instance]] = {"train": train_data}

    validation_data_path = params.get('validation_data_path', None)
    if validation_data_path is not None:
        logger.info("Reading validation data from %s", validation_data_path)
        validation_data = validation_and_test_dataset_reader.read(validation_data_path)
        datasets["validation"] = validation_data

    test_data_path = params.pop("test_data_path", None)
    if test_data_path is not None:
        logger.info("Reading test data from %s", test_data_path)
        test_data = validation_and_test_dataset_reader.read(test_data_path)
        datasets["test"] = test_data

    return datasets
datasets = datasets_from_params(params)

[autoreload of src.dataset_readers.vctk failed: Traceback (most recent call last):
  File "/home/lnan6257/work/allennlp-speech/myenv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/lnan6257/work/allennlp-speech/myenv/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/home/lnan6257/work/allennlp-speech/myenv/lib/python3.6/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/lnan6257/work/allennlp-speech/myenv/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 608, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
  File "/home/lnan6257/work/allennlp-speech/src/dataset_readers/vctk.py", line 20, in <mod

instantiating class <class 'allennlp.data.dataset_readers.dataset_reader.DatasetReader'> from params {} and extras {}


ConfigurationError: 'key "type" is required at location "dataset_reader."'